In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
# from tensorflow.keras.applications.efficientnet import EfficientNetB4,EfficientNetB0,EfficientNetB7
# from tensorflow.keras.applications.efficientnet import preprocess_input
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
# from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2
# from tensorflow.keras.applications.resnet import ResNet50, ResNet101, ResNet152
# from tensorflow.keras.applications.nasnet import NASNetMobile
# from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
# from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.applications.xception import Xception
# from tensorflow.keras.applications.densenet import DenseNet121,DenseNet169,DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import random
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from gradcam import VizGradCAM


In [ ]:
# mannequin_path = os.path.join('../','eventsData/training/mannequin')
# # not_mannequin_path = os.path.join('../','not_mannequin_filter')

# mannequin_img_path = [os.path.join(mannequin_path,x) for x in os.listdir(mannequin_path)]
# # not_mannequin_img_path = [os.path.join(not_mannequin_path,x) for x in os.listdir(not_mannequin_path)]

# print(mannequin_img_path[:5])
# # print(not_mannequin_img_path[:5])

# random.shuffle(mannequin_img_path)
# random.shuffle(mannequin_img_path)
# # random.shuffle(not_mannequin_img_path)
# # random.shuffle(not_mannequin_img_path)

# print(mannequin_img_path[:5])
# print(not_mannequin_img_path[:5])

In [ ]:
# train_mannequin_img, test_mannequin_img = train_test_split(mannequin_img_path,test_size = 0.20,random_state=42)

In [ ]:
train_datagen = IDG(rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True,shear_range=0.2,zoom_range=0.2,fill_mode='nearest')
val_datagen = IDG()
test_datagen = IDG()

train_path = '../EventsFrame_19012020/upscaled_data_TrainTestVal/training'
val_path = '../EventsFrame_19012020/upscaled_data_TrainTestVal/validation'
test_path = '../EventsFrame_19012020/upscaled_data_TrainTestVal/testing'
shape = (224,224,3)
SHAPE = (224,224,3)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical',shuffle=True,color_mode='rgb',classes=['mannequin'])
val_generator = val_datagen.flow_from_directory(val_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical',shuffle=True,color_mode='rgb',classes=['mannequin','not_mannequin'])
test_generator = test_datagen.flow_from_directory(test_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical')


In [ ]:
# a,b = train_generator.next()

In [ ]:
# a[1].max()

In [ ]:
# plt.imshow(a[5])

In [ ]:
# VizGradCAM(MobileNetV2(weights='imagenet'),a[4])

In [ ]:

def get_model(train=True):
    
    pre_process = Lambda(preprocess_input)
    vgg = VGG16(weights = 'imagenet', include_top = True, input_shape = SHAPE)
    vgg = Model(vgg.input, vgg.layers[-1].output)
    vgg.trainable = False
    
    inp = Input(SHAPE)
    vgg_16_process = pre_process(GaussianNoise(0.1)(inp))
    vgg_out = vgg(vgg_16_process)
    
    noise = Lambda(tf.zeros_like)(vgg_out)
    noise = GaussianNoise(0.1)(noise)

    if train:
        x = Lambda(lambda z: tf.concat(z, axis=0))([vgg_out,noise])
        x = Activation('relu')(x)
    else:
        x = vgg_out
        
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(2, activation='softmax')(x)

    model = Model(inp, out)
    model.compile(Adam(lr=1e-4), loss='binary_crossentropy')
    
    return model

In [ ]:
def wrap_generator(generator):
    
    while True:
        x,y = next(generator)
        y = tf.keras.utils.to_categorical(y)
        zeros = tf.zeros_like(y) + tf.constant([1.,0.])
        y = tf.concat([y,zeros], axis=0)
        
        yield x,y

In [ ]:
model = get_model()

In [ ]:
model.summary()

In [ ]:
# efficientB2_model = VGG16(include_top=True,weights='imagenet',input_shape=shape)
# efficientB2_model.trainable=False
# x = GlobalAveragePooling2D()(efficientB2_model.output)
# x = Dense(512,activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(128,activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(2,activation='softmax')(x)

# model = Model(efficientB2_model.input,x)


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),metrics=['accuracy'])

In [ ]:
history=model.fit(wrap_generator(train_generator),steps_per_epoch=train_generator.samples/train_generator.batch_size,epochs=10)

In [ ]:
val_generator = val_datagen.flow_from_directory(val_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical',shuffle=True,color_mode='rgb',classes=['mannequin','not_mannequin'])

In [ ]:
test_num = val_generator.samples

label_test = []
for i in range((test_num // val_generator.batch_size)+1):
    X,y = val_generator.next()
    label_test.append(y)
        
label_test = np.argmax(np.vstack(label_test), axis=1)
label_test.shape

In [ ]:

### SWITCH TO INFERENCE MODE TO COMPUTE PREDICTIONS ###

ineference_model = get_model(train=False)
ineference_model.set_weights(model.get_weights())

In [ ]:
pred_test = np.argmax(ineference_model.predict(val_generator), axis=1)

In [ ]:
print('ACCURACY:', accuracy_score(label_test, pred_test))

In [ ]:
model.save('ResNet50V2_softmax_Synthetic_Upscale_Frame.h5')

In [ ]:
tf.saved_model.save(model,"mobilenetV2_softmax_GM_rescale")

In [ ]:
# from tensorflow.keras.models import load_model
# model_import = load_model('mobilenetV2.h5')